In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Note: TensorFlow is not needed for PySoap to work. It's only used to load the dataset
import tensorflow as tf

import PyNetwork

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(*x_train.shape, 1) / 255
x_test = x_test.reshape(*x_test.shape, 1) / 255

labels = np.eye(10)
y_train = labels[y_train]
y_test = labels[y_test]

In [26]:
model = PyNetwork.Sequential()

model.add( PyNetwork.layers.Input((28, 28, 1)) )
model.add( PyNetwork.layers.Conv_2D(3, (3, 3), 1, activation_function='relu') )
model.add( PyNetwork.layers.Flatten() )
model.add( PyNetwork.layers.Dense(100, activation_function='relu', l2=0.0, l1=0.001, 
                                  trainable_mask=np.random.randint(0, 2, size=(100, 2028))) )
model.add( PyNetwork.layers.BatchNorm() )
model.add( PyNetwork.layers.Dense(10, activation_function='softmax') )

# optimizer = PySoap.optimizers.SGD(learning_rate=0.01, momentum=0.001)
model.build(loss_function='cross_entropy', optimizer="adam", metrics = 'accuracy')

In [27]:
model.layers

{1: <PyNetwork.layers.Input.Input at 0x29a2d50f0>,
 2: <PyNetwork.layers.Convolutional.Conv_2D at 0x29717b9a0>,
 3: <PyNetwork.layers.Flatten.Flatten at 0x29a292c20>,
 4: <PyNetwork.layers.Dense.Dense at 0x29a2d4f40>,
 5: <PyNetwork.layers.BatchNorm.BatchNorm at 0x29a293280>,
 6: <PyNetwork.layers.Dense.Dense at 0x29a292b30>}

In [28]:
original_weights = model.layers[4].W.copy()
mask = model.layers[4].trainable_mask.copy()

In [29]:
model.summary()

Input                 :    Input Shape  (None, 28, 28, 1)
Conv 2D 3 x (3, 3)    :    Output Shape (None, 26, 26, 3)
Flatten               :    Output Shape (None, 2028)
Dense (100,)          :    Output Shape (None, 100)
Batch Norm            :    Output Shape (None, 100)
Dense (10,)           :    Output Shape (None, 10)


In [30]:
%%time
model.train(x_train, y_train, epochs=3, batch_size=128, verbose=True)

Training on 60000 samples
Epoch 1/3
cross_entropy: 0.8958 - accuracy: 0.7188
Training on 60000 samples
Epoch 2/3
cross_entropy: 0.8244 - accuracy: 0.7266
Training on 60000 samples
Epoch 3/3
cross_entropy: 1.0565 - accuracy: 0.6797
CPU times: user 2min 10s, sys: 46.3 s, total: 2min 56s
Wall time: 18.1 s


In [31]:
model.evaluate(x_test, y_test)

'cross_entropy: 0.7261 - accuracy: 0.7736'

In [32]:
new_weights = model.layers[4].W.copy()

In [33]:
# The non-trainable weights stay the same through out training
np.all(original_weights[~mask] == new_weights[~mask])

True